# That 70-80% of my Job

Basically, get data, make it into a usable format where I have my `X`s and `Y`s ready to fit into my desired deep learning framework. And as we all know it's the ultimate Sequence-to-Sequence (aka. Encoder-Decoder) framework.

# Scenario / Background

I just got a request from my customer (aka my students) to demonstrate how to do NLP work with twitter data and somewhat force the topic to be related to sentiment analysis. 

But first to fit anything into my Seq2Seq framework I need to have my Xs and Ys. One way to do thing is I start digging around datasets that already have pre-defined labels and I use them. Fine, they're meh... 

So lets do what I do in that 10% of my time, i.e. **craft the project requirements**. 


# What is the ultimate task that I want to solve?

This is the discussion between your managers and the customers. 

**Hypothesis 1** (from customers): I can somehow create sentiment labels of tweets and create some labels type so that my customers can understand their social media's post. 


**Reviewer 2** (the devils advocate): `ddddaaaaairrr...`  Wrong, actually, not fully wrong but that's not a real task, that's some fake imagination of the customers, so I'll do some expectation management. 

**Hypothesis 2** (your manager's counter proposal to customer): Why don't I build you something that can detect one label first and then we can move on to more when you're satisfied. What's your greatest pain-point? Perhaps detecting some post that negatively affects the company?


**Hypothesis 2** (your support for the proposal): (thinking in your head, you know what's the worst that can occur for @smrt_singapore) How about I help you detect posts that relates to delays? That'll be a good start. 


**Hypothesis 2** (finalize the project requirements, for this component): So for the first component, lets try to take tweets and just detect whether it's a tweet relating to delays. 


### Hopefully customers, think it's reasonable and see how you can help deploy this first AI solution. 

# Lets start coding...  But first how to get the data?! ちょっと待って!!

**What's my `X`s? Tweets. **

**What's my `Y`s? I don't have `Y`s!!! **

WTH... What did my manager just agreed on?! (Actually, all these while you're also in the conversation but you don't sound out and customer agrees... LOLOL)


Alright, lets see what tweets are available when I query `@smrt_singapore` and `delay`.


# Google "twitter download data python"

And somehow that leads me to https://blog.usejournal.com/scraping-50-million-tweets-for-5-in-5-hours-9dca3031a734

Nice!! That's $5 for quite a lot of data. Lets try that.

# Exploratory Data Analaysis

In [3]:
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
import json

In [4]:
def twitter_url(keywords, lang='en', since=None, until=None):
    """
    :param keywords: List of keywords to query.
    :type keywords: list(str)
    :param lang: Language, two char ISO language code.
    :type lang: str
    :param since: Start date in YYYY-MM-DD format.
    :type since: str
    :param until: End date in YYYY-MM-DD format.
    :type until: str
    """
    # Base URL of twitter search.
    base_url = 'https://twitter.com/search?f=tweets&vertical=default&q={q}'
    # Form the query.
    keywords_str = ' '.join(keywords)
    query =  '{} since:{} until:{}'.format(keywords_str, since, until)
    # Escaping for URL.
    query = query.replace(' ', '%20').replace('#', '%23').replace(':', '%3A').replace('@', '%40')
    # Create the final query with query and lang.
    return base_url.format(q=query, lang=lang)

    
my_user_agent = str('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/72.0.3626.121 Safari/537.36')
headers = {'User-Agent': my_user_agent}


In [5]:
# Generate the URL.
url = twitter_url(['@smrt_singapore', 'delay'], 
                  since='2010-01-01', until='2020-01-20')
print(url)

https://twitter.com/search?f=tweets&vertical=default&q=%40smrt_singapore%20delay%20since%3A2010-01-01%20until%3A2020-01-20


In [8]:
reponses = requests.get(url, headers=headers).content.decode('utf8')
bsoup = BeautifulSoup(reponses, 'lxml')

# After looking at the web-structure of the page, I find that tweets are in 
# <li data-item-type="tweet"> ... </li>
tweets = bsoup.find_all("li", {"data-item-type": "tweet"})

In [9]:
tweets[0]

<li class="js-stream-item stream-item stream-item" data-item-id="1199640194015645697" data-item-type="tweet" id="stream-item-tweet-1199640194015645697">
<div class="tweet js-stream-tweet js-actionable-tweet js-profile-popup-actionable dismissible-content original-tweet js-original-tweet has-cards has-content" data-component-context="tweet" data-conversation-id="1199640194015645697" data-disclosure-type="" data-follows-you="false" data-has-cards="true" data-item-id="1199640194015645697" data-mentions="SMRT_Singapore" data-name="MRT Singapore Service Information" data-permalink-path="/mrtsginfo/status/1199640194015645697" data-reply-to-users-json='[{"id_str":"932838812119527424","screen_name":"mrtsginfo","name":"MRT Singapore Service Information","emojified_name":{"text":"MRT Singapore Service Information","emojified_text_as_html":"MRT Singapore Service Information"}},{"id_str":"307781209","screen_name":"SMRT_Singapore","name":"SMRT Corporation","emojified_name":{"text":"SMRT Corporation

# Banzai!! I get what I need =)

Actually to do proper EDA, it looks like have to do some data munging first, so lets **Extract, Transform, Load** before we come back to **EDA** then we repeat the cyle of `EDA-> ETL -> EDA -> ETL` until we can define our `X`s and `Y`s. 


# Extract Transform Load

In [10]:
# Further inspection, says I can 
# 1. skip <a class=js-action-profile-promoted"> ... </a>
# 2. look for the tweet text in <p class="tweet-text"> ... </p>
# 3. find the timestamp in <span class="_timestamp" "data-time-ms"="..." />
# 4. the "replies" counts for this tweet are in
#   <div class="ProfileTweet-action--reply"> 
#        <span class="ProfileTweet-actionCountForPresentation"> ... </span>
#   </div>
# 5. the "retweets" counts for this tweet are in 
#   <div class="ProfileTweet-action--retweet"> 
#        <span class="ProfileTweet-actionCountForPresentation"> ... </span>
#   </div>
# 6. the "favorite"/ "heart"/"like" counts for this tweet are in 
#   <div class="ProfileTweet-action--favorite"> 
#        <span class="ProfileTweet-actionCountForPresentation"> ... </span>
#   </div>
# 7. the "verified" twitter account icon is in <span class="Icon--verified"> ... </span>
# 8. the permalink to the tweet is found in <div data-permalink-path="...">
#
# Of course, I can do this by looking at the HTML but actually the full code
# linked to the blog post shows these at https://gist.github.com/heffo42/caaef8f2c2d32757c2247cad4714695d#file-main-py-L24


def is_promoted_tweet(tweet_li):
    """
    Check if tweet is promoted.
    :param tweet_li: The results of bsoup.find_all("li", {"data-item-type": "tweet"})
    :type tweet_li: bs4.element.Tag
    """
    return True if tweet_li.find("a", {"class" : "js-action-profile-promoted"}) else False


def munge_tweet_meta(tweet_li, div_class):
    """
    Check counts of meta data, i.e. retweets, replies and likes. 
    """
    data_block = tweet_li.find("div", {"class": div_class})
    data = data_block.find("span", {"class" : "ProfileTweet-actionCountForPresentation"}).get_text()
    return 0 if data == '' else int(data)


def is_user_verified(tweet_li):
    return True if tweet_li.find("span", {"class" : "Icon--verified"}) != None else False

def munge_tweet_li(tweet_li):
    perma_path = tweet_li.find('div').attrs['data-permalink-path']
    text = tweet_li.find("p", {"class" : "tweet-text"}).get_text()
    date = tweet_li.find("span", {"class" : "_timestamp"})["data-time-ms"]
    reply_count = munge_tweet_meta(tweet_li, "ProfileTweet-action--reply")
    fav_count = munge_tweet_meta(tweet_li, "ProfileTweet-action--favorite")
    retweet_count = munge_tweet_meta(tweet_li, "ProfileTweet-action--retweet")
    # Collate all the key-value pairs.
    _json = {"text" : text, "date" : date,
             "num_replies": reply_count, "num_retweets": retweet_count,
             "num_fav": fav_count, "verified" : is_user_verified(tweet_li),
             "promoted": is_promoted_tweet(tweet_li), 'perma_path': perma_path
            }
    return _json
    
    

In [11]:
munge_tweet_li(tweets[0])

{'text': 'We are receiving news from commuters, stating that the North South Line is running with Delays right now\n\nDo expect some additional travelling time on the North South Line, due to a Faulty Track at Marsiling Northbound\n\n@SMRT_Singapore pic.twitter.com/kRgXDEprKJ',
 'date': '1574851485000',
 'num_replies': 0,
 'num_retweets': 0,
 'num_fav': 0,
 'verified': False,
 'promoted': False,
 'perma_path': '/mrtsginfo/status/1199640194015645697'}

In [13]:
# Lets look at the first 10 tweets.
for t in tweets[:10]:
    print(munge_tweet_li(t))
    print()

{'text': 'We are receiving news from commuters, stating that the North South Line is running with Delays right now\n\nDo expect some additional travelling time on the North South Line, due to a Faulty Track at Marsiling Northbound\n\n@SMRT_Singapore pic.twitter.com/kRgXDEprKJ', 'date': '1574851485000', 'num_replies': 0, 'num_retweets': 0, 'num_fav': 0, 'verified': False, 'promoted': False, 'perma_path': '/mrtsginfo/status/1199640194015645697'}

{'text': "Yeah gonna guess it's something to do with a delay pic.twitter.com/uhn02vC1ub", 'date': '1573087367000', 'num_replies': 0, 'num_retweets': 0, 'num_fav': 0, 'verified': False, 'promoted': False, 'perma_path': '/kkyz13/status/1192240949604179968'}

{'text': '@SMRT_Singapore Train service delay from OTP NSEW,  toward Tuas Link? Dwell time more than 5 mins every station.', 'date': '1572103075000', 'num_replies': 0, 'num_retweets': 0, 'num_fav': 0, 'verified': False, 'promoted': False, 'perma_path': '/AriTumijo/status/1188112528443727877'}


# Ah ha!! Any tweet with "delay" can be treated as my positive examples!!!

Having the "delay" in the query seems to be a good proxy to detect the "positive" examples of identifying the "delay" related tweets =)

In [14]:
print(url)

https://twitter.com/search?f=tweets&vertical=default&q=%40smrt_singapore%20delay%20since%3A2010-01-01%20until%3A2020-01-20


# Previously, the above URL only gets us the first page of the search query, now lets get all pages.

In [10]:
# It looks like there's some sort of search pointers that are non integers
# to prevent people from just loop through the pages by doing += 1 to some 
# variables, so it's some hashes hidden in <div class="stream-container" ... > ... </div>
bsoup.find("div", {"class": "stream-container"})["data-min-position"]

'thGAVUV0VFVBaGgKfNtda27h4WgsCq6dCZ_aUhEjUAFQAlAFUAFQAA'

In [20]:
# Given the above knowledge, I can parse through the pages like this.

def fetch_tweets(query, lang):
    # Generate the URL.
    url = twitter_url(query, lang, since='2010-01-01', until='2020-01-20')
    reponse = requests.get(url, headers=headers).content.decode('utf8')
    bsoup = BeautifulSoup(reponse, 'lxml')

    # After looking at the web-structure of the page, I find that tweets are in 
    # <li data-item-type="tweet"> ... </li>
    tweets = bsoup.find_all("li", {"data-item-type": "tweet"})

    tweet_jsons = [munge_tweet_li(t) for t in tweets]

    # Then lets iterate through an arbitary no. of pages.
    max_pages = 10000
    # We fetch that hash pointer that was pointing to the next batch of tweets.
    next_pointer = bsoup.find("div", {"class": "stream-container"})["data-min-position"]

    # Now, we find that the next page of tweets URL is not exactly the same as before.
    # It requires (i) hash of the next pointer, (ii) query and (iii) lang.
    next_tweet_page_url = str('https://twitter.com/i/search/timeline?f=tweets&vertical=' 
                              'default&include_available_features=1&include_entities=1&' 
                              'reset_error_state=false&src=typd&max_position={pos}&q={q}&l={lang}')

    # Iterate through the max no. of pages.
    for i in tqdm(range(max_pages)):
        tweets = []
        url = next_tweet_page_url.format(q=' '.join(query), lang=lang, pos = next_pointer)
        ##print(url)
        # Fetch the URL
        response = requests.get(url, headers=headers)
        # Do some checks to make sure that the requests gets the page.
        # Then just skips it if it's not successful, i.e. status_code != 200
        if response.status_code != 200:
            continue 
        # This is interesting because no the timeline outputs itself, actually
        # iterates through the timeline of the queries rather than the original
        # search results. So we can immediate try to get the json object 
        # that stores the html we need to parse.
        reponse_content = response.content.decode('utf8')
        try:
            json_resp = json.loads(reponse_content)
            html = json_resp['items_html']
        except: # Skip pages that we can't parse.
            continue
        # Similar to the first page, we look for the tweets.
        bsoup = BeautifulSoup(html, 'lxml')
        tweets = bsoup.find_all("li", {"data-item-type": "tweet"})
        if tweets: # Munge it the same way.
            munged_tweets = [munge_tweet_li(t) for t in tweets]
            ##print(munged_tweets)
            tweet_jsons.extend(munged_tweets)
        # If there's no more next items to load, just exit the loop

        if (not json_resp['has_more_items']) and (json_resp["new_latent_count"] == 0):
            break
        # Find and update the next_pointer for the next page to load.
        next_pointer = json_resp['min_position']
    return tweet_jsons

In [21]:
lang='en'
query = ['@smrt_singapore', 'delay']
smrt_delay_tweets = fetch_tweets(query, lang)

  1%|          | 106/10000 [01:16<1:46:42,  1.55it/s]

In [22]:
with open('smrt_delay_tweets.json', 'w') as fout:
    json.dump(smrt_delay_tweets, fout)

In [ ]:
len(smrt_delay_tweets)

In [ ]:
lang='en'
query = ['@smrt_singapore']
smrt_tweets = fetch_tweets(query, lang)

In [23]:
with open('smrt_delay_tweets.json', 'w') as fout:
    json.dump(smrt_tweets, fout)

NameError: name 'smrt_tweets' is not defined

In [14]:
len(smrt_tweets)

125900

# Hmmmm, there seems to be something "weird" with some of the tweets. 

In [ ]:
smrt_tweets[101]

In [26]:
smrt_tweets[10001]

{'text': "I'm at Orchard MRT Station (NS22) - @smrt_singapore in Singapore https://www.swarmapp.com/c/lV8z0i0mc48\xa0",
 'date': '1579418258000',
 'tweetId': '1218794628167667712',
 'num_replies': 0,
 'num_retweets': 0,
 'num_fav': 0,
 'verified': False,
 'promoted': False}